### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Spring 2018 ###

# Megapolis #

#### Sam Berglin(berglin@wisc.edu), Aiden Song (aiden@cs.wisc.edu), Rachel Sowada(rsowada@wisc.edu), and Andrew Eng(ateng@wisc.edu)

*****

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-model)
1. [Solution](#3.-Solution)
1. [Results and Discussion](#4.-Results-and-discussion)
  1. [Optional Subsection](#4.A.-Feel-free-to-add-subsections)
1. [Conclusion](#5.-Conclusion)

## 1. Introduction ##

The first few sentences should give a quick overview of the entire project. Then, elaborate with a description of the problem that will be solved, a brief history (with [citations](https://en.wikipedia.org/wiki/Citation)) of how the problem came about, why it's important/interesting, and any other interesting facts you'd like to talk about. You should address and explain where the problem data is coming from (research? the internet? synthetically generated?) Also give an outline of the rest of the report.

This section should be 300-600 words long, and **should be accessible to a general audience** (don't assume your reader has taken the class!). Feel free to include images if you think it'll be helpful:

![fixit flowchart][flow]

For more help on using Markdown, see [this reference](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet).

[flow]: https://s-media-cache-ak0.pinimg.com/736x/f5/75/c5/f575c53b93724808c6f0211890a54900.jpg

## 2.1 Toy Mathematical Model ##

Our model can be represented as the mixed integer program (MIP),  
<br/>

<center>
$\begin{equation*}
\begin{aligned}
& \underset{x}{\text{maximize}}
& & x^T(P + A) \\
& \text{subject to}
& & x \geq p \cdot X_{min} \\
&&& B \geq x^TC \\
\end{aligned}
\end{equation*}$
</center>
<br/>
  
  In our model, $x = (x_1, x_2, ..., x_t)$ represents a vector of the number of each of the t types of buildings possible. Clearly, each of these values are integers.  
  
  $P, A, \text{and } C$ also represent similar quanitities, where $P = (p_1, p_2, ... p_n)$ and similar for $A$ and $C$. They are also t-dimensional vectors with the ith entry corresponding to the ith type of building. For a given building type i, $c_i$ represents the cost of constructing a building, $p_i$ represents the annual profit, and $a_i$ represents the attractiveness. In our simple model, we measure each of these for a building on a scale from -10 to 10. Negative numbers represent cost rather than profit, and "ugliness" rather than attractiveness (note: there cannot be a negative cost for constructing a building).  
  
  $X_{\text{min}}$ represents the minimum per person requirements for each building. Like $x$, it is t-dimensional for each of the minimum requirements for the t buildings. For clarity, we represent $X_{\text{min}}$ in number of buildings per 1,000 people. Furthermore, $p$ represents the number of people or population the city must be able to support.
  
  Since our decision variables (the number of buildings) must be positive integers, the problem is a MIP. In our model, we essentially are trying the maximize the annual "value" of a city given a certain budget for **completely** constructing it.

# 2.2 Zoning  
  
  Obviously, choosing what buildings to build is not the entirety of how buildings are planned. We must choose where to place them, while still maintaining Level I's requirements.
  
  Consider the possible "world" to be an nxn board called $W$. $W[i,j]$ corresponds to the type of building built at location $[i,j]$ of the world. Given T types of possible buildings, we can have a one hot encoded vector $w_{i,j}=(0,0,..., 0, 1, 0, ..., 0)$ at each $W[i,j]$. The 1 at position $k$ of $w_{i,j}$ indicates that building type $k$ is built at $i,j$. We denote position $k$ of $w_{i,j}$ as $w_{i,j}^k$. We still have a similar $P$ and $A$ matrices. We represent our problem as below. We will explain the purpose of each constraint and notation afterwards.
  
<br/>
<center>
$\begin{equation*}
\begin{aligned}
& \underset{w_{i,j} \in W}{\text{maximize}}
& & \sum_{w_{i,j} \in W}t^T(P + A) \\
& \text{subject to}
& &  \forall b_{i,j} \in B, w_{i,j} \in W_\text{base} : w_{i,j} = b_{i,j} \space\space (A)\\
&&& \sum_{w_{i,j} \in W} w_{i, j} \geq X_\text{min} \cdot p \div 1000  \space\space (B)\\
&&& \sum_{w_{i,j} \in W} t^T C \leq B \space\space (C)\\
&&& \forall w_{i,j} \in W \sum_{1 = k}^{T} w_{i,j}^k \leq 1 \space\space (D)\\
&&& \forall w_{i,j} \in W_\text{inner} : w_{i,j} \leq w_{i-1,j} + w_{i,j-1} + w_{i+1,j} + w_{i,j+1} \space\space (E)\\
&&& \sum_{w_{i,j} \in W} (4w_{i,j} - w_{i-1,j} - w_{i+1,j} - w_{i,j+1} - w_{i,j-1}) \leq R \cdot \sum_{w_{i,j} \in W} w_{i,j} \space\space (F)\\
\end{aligned}
\end{equation*}$
</center>
<br/>
<br/>

  (A) This constraint ensures that the base city exists in the middle of the map and that the outside (or border) of the world is empty. The base city exists so that the next iteration can grow from it, and the border being empty aids in perimeter calculation in constraint F.
  
  (B) This constraint ensures that the population is suported by the proper number of buildings. It is the same as the toy example.
  
  (C) This constraint ensures that the budget is respected. It is the same as the toy example.
  
  (D) This constraint ensures that each vector $w_{i,j}$ can have at most one 1 and the remainder are zeros. This ensures that there is only one "type" of building assigned to each square of the world.
  
  (E) This is the first nontrivial constraint. It causes a phenomenon of zoning. Zoning is where certain areas of a community are reserved for certain types of buildings. For example, there can be residential zones or business zones. In order to implement this into our example, we ensure that each type of buildings must form a **continguous** area. This is represented in the constraint by not letting type $k$ of a building be built **unless** there is one of type $k$ surrounding it. $W_\text{inner}$ represents the inner world (i.e. the world without the border). We only sum over these squares since the border is defined to be zero.
  
  (F) This constraint builds upon constraint (E). As a whole, it prevents regions from being strangley shaped by ensuring a proper ratio of perimeter to area. The expression on the left calculates a T dimensional vector representing the perimeter of each of the contiguous building regions. The right side's summation calculates the area of the continguous building regions. The corresponding ratio is represented by $R$.

# Zoning Code 

In [91]:
# data
building_types = ["business", "hospital", "park", "farm", "school", "home"]
num_types = length(building_types)
# we converted profits, and attractiveness on a scale from -10 to 10
cost = [20,100,30,20,60,4]
profits = [9,-3,-1,6,-4,1]
attractiveness = [4,-1,3,-5,6,2]
min_num_building_per_1kPeople = [5,1,3,1,1,10]
N = 6
numType = size(building_types,1)

base = zeros(N,N,numType)
base[1,5,5] = 1
base[6,1,5] = 1
base[5,5,6] = 1
base[2,3,1] = 1
#= base = [
    [0 0 0 0 0 0], [0 0 0 0 0 0], [0 0 0 0 0 0], [0 0 0 0 0 0], [0 0 0 0 1 0], [0 0 0 0 0 0],
    [0 0 0 0 0 0], [1 0 0 0 0 0], [0 0 0 0 0 0], [0 0 0 0 0 0], [0 0 0 0 0 0], [0 0 0 0 0 0],
    [0 0 0 0 0 0], [0 0 0 0 0 0], [0 0 0 0 0 0], [0 0 0 0 0 0], [0 0 0 0 0 0], [0 0 0 0 0 0],
    [0 0 0 0 0 0], [0 0 0 0 0 0], [0 0 0 0 0 0], [0 0 0 0 0 0], [0 0 0 0 0 0], [0 0 0 0 0 0],
    [0 0 0 0 0 0], [0 0 0 0 0 0], [0 0 0 0 0 0], [0 0 0 0 0 0], [0 0 0 0 0 1], [0 0 0 0 0 0],
    [0 0 0 0 1 0], [0 0 0 0 0 0], [0 0 0 0 0 0], [0 0 0 0 0 0], [0 0 0 0 0 0], [0 0 0 0 0 0],
]
=#
;

In [92]:
using JuMP, Gurobi
# assume that we have population of 10,000 people
population = 10
# assume that we have a budget of 
budgets = 1000

m = Model(solver = GurobiSolver(OutputFlag = 0))
@variable(m, x[1:N, 1:N, 1:numType], Bin)

# can't have more than one building in a spot
@constraint(m, flow[i in 1:N, j in 1:N], sum(x[i,j,:]) <= 1)  

# buildings in the base must stay (for now)
@constraint(m, flow1[i in 1:N, j in 1:N, k in 1:numType],x[i,j,k] >= base[i,j,k])

# meet the minimum number per building
@constraint(m, flow2[i in 1:numType], sum(x[:,:,i]) >= min_num_building_per_1kPeople[i])

# Stay within the given budget 
@constraint(m, sum((sum(x[:,:,k]) - sum(base[:,:,k]))*cost[k] for k in 1:numType) <= budgets)  

# Zoning constraint 1 
@constraint(m, flow3[i in 2:N-1, j in 2:N-1, k in 1:numType], x[i,j,k] <= x[i-1,j-1,k] + x[i-1,j+1,k] + x[i+1,j-1,k] + x[i+1,j+1,k])

# TO DO Zoning Constraint 2 


@expression(m, totalProfit, sum(sum(x[:,:,k])*profits[k] for k in 1:numType))
@expression(m, totalAttractiveness, sum(sum(x[:,:,k])*attractiveness[k] for k in 1:numType))

@objective(m, Max, totalProfit + totalAttractiveness)
status = solve(m)
printMap(getvalue(x), N,N,numType)


Academic license - for non-commercial use only
park	hospital	home	park	school	home	
home	home	business	home	business	home	
business	business	business	business	home	business	
business	business	business	business	business	home	
home	business	business	business	home	business	
school	business	business	business	park	farm	


In [89]:
function printMap(map, N, M, P)
    
    for i =1:N
       for j = 1:M
            for k = 1:P
                if(map[i,j,k] == 1)
                    print(building_types[k])
                    break; 
                end
            end
            print("\t")
        end
        print("\n")
    end
    
end

printMap (generic function with 1 method)

## 3. Solution ##

Here, you should code up your model in Julia + JuMP and solve it. Your code should be clean, easy to read, well annotated and commented, and it should compile! You are not allowed to use other programming languages or DCP packages such as `convex.jl`. **I will be running your code**. I suggest having multiple code blocks separated by text blocks that explain the various parts of your solution. You may also solve several versions of your problem with different models/assumptions.

It's fine to call external packages such as `Gurobi`, but try to minimize the use of exotic libraries.

In [1]:
# data
building_types = ["business", "hospital", "park", "farm", "school", "home"]
num_types = length(building_types)
# we converted profits, and attractiveness on a scale from -10 to 10
cost = [20000,100000,3000,2000,60000,400]
profits = [9,-3,-1,6,-4,1]
attractiveness = [4,-1,3,-5,6,2]
min_num_building_per_1kPeople = [5,1,3,1,1,333]
;

In [2]:
using JuMP, Gurobi

m = Model(solver=GurobiSolver(OutputFlag=0))

# assume that we have population of 10,000 people
population = 10
# assume that we have a budget of 
budgets = 10000000

@variable(m, buildings[1:6],Int)
@constraint(m,var[i = 1:6], buildings[i] >= min_num_building_per_1kPeople[i] * population)
@constraint(m, dot(cost,buildings) <= budgets)
@objective(m, Max,buildings'*(profits + attractiveness))

s = solve(m)
println()
# get the values of the number of buildings vector
building = getvalue(buildings)
for b = 1:6
    println("Number of ",building_types[b]," to build: ", building[b])
end
println("At the budgets of \$", budgets, " the objective would be ", getobjectivevalue(m))

Academic license - for non-commercial use only

Number of business to build: 50.0
Number of hospital to build: 10.0
Number of park to build: 30.0
Number of farm to build: 10.0
Number of school to build: 10.0
Number of home to build: 18225.0
At the budgets of $10000000 the objective would be 55375.0


Remember to make sure your code compiles! I will be running your code!

## 4. Results and discussion ##

Here, you display and discuss the results. Show figures, plots, images, trade-off curves, or whatever else you can think of to best illustrate your results. The discussion should explain what the results mean, and how to interpret them. You should also explain the limitations of your approach/model and how sensitive your results are to the assumptions you made.

Use plots (see `PyPlot` examples from class), or you can display results in a table like this:

| Tables        | Are           | Cool  |
| ------------- |:-------------:| -----:|
| col 3 is      | right-aligned |\$1600 |
| col 2 is      | centered      |  \$12 |
| zebra stripes | are neat      |   \$1 |

### 4.A. Feel free to add subsections

#### 4.A.a. or subsubsections

## 5. Conclusion ##

Summarize your findings and your results, and talk about at least one possible future direction; something that might be interesting to pursue as a follow-up to your project.